In [1]:
# -*- coding: utf-8 -*-
"""
Created on Thu Dec 19 17:08:19 2024

@author: Rodrigo Meira
"""

from eos_database import *
from compressor_class import *
from compression import *
from gc_eos_soave import *
from casadi import *
import numpy as np
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
from viscosity import *
from plenum_system import *

In [2]:
list_names = ["CH4", "C2H6", "C3H8", "iC4H10", "nC4H10", "iC5H12", "nC5H12", 
                  "nC6H14", "nC7H16", "nC8H18", "nC9H20", "nC10H22", "nC11H24", 
                   "nC12H26", "nC14H30", "N2", "H2O", "CO2", "C15+"]

nwe = [0.9834, 0.0061, 0.0015, 0.0003, 0.0003, 0.00055, 0.0004, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0003, 0.0, 0.0008, 0.0]

dict_composition= {list_names[i]: nwe[i] for i in range(len(nwe))}

mixture = Mixture(list_of_species, dict_composition)

volumn_desviation = [0] * 19

vlv = valve(kv=0.38)
Vpp = 2.0 
Lc = 2.0 
A1 = 2.6e-3
gas = gc_eos_class(mixture, 300, 4500, None, 1, 0, Aij, volumn_desviation, 'gas')
comp = CompressorClass()
visc = viscosity(mixture, volumn_desviation)
compressor = compression(gas, comp, visc)
plenum_sys = plenum(gas, compressor, vlv, Vpp, Lc, A1)

In [10]:
n_x = 3
n_z = 11

def system_residuals(y, u0, plenum_sys):
    # Separa as variáveis
    x = y[:n_x]
    z = y[n_x:]
    
    # Substitua as expressões abaixo pelas suas equações do modelo em estado estacionário.
    ode_sym, alg_sym = plenum_sys.evaluate_dae(None, x, z, u0)
    
    res_ode = np.zeros(n_x)
    res_ode[0] = x[0] - ode_sym[0].item()
    res_ode[1] = x[1] - ode_sym[1].item()
    res_ode[2] = x[2] - ode_sym[2].item()
    
    # Substitua os exemplos abaixo pelas equações algébricas reais do seu sistema.
    res_alg = np.zeros(n_z)
    res_alg[0]  = z[0]  - alg_sym[0].item()  
    res_alg[1]  = z[1]  - alg_sym[1].item()
    res_alg[2]  = z[2]  - alg_sym[2].item()
    res_alg[3]  = z[3]  - alg_sym[3].item()
    res_alg[4]  = z[4]  - alg_sym[4].item()
    res_alg[5]  = z[5]  - alg_sym[5].item()
    res_alg[6]  = z[6]  - alg_sym[6].item()
    res_alg[7]  = z[7]  - alg_sym[7].item()
    res_alg[8]  = z[8]  - alg_sym[8].item()
    res_alg[9]  = z[9]  - alg_sym[9].item()
    res_alg[10] = z[10] - alg_sym[10].item()

    res = np.concatenate((res_ode, res_alg))
    return res

def compute_steady_state(u0, gas, compressor, valve, plenum_sys, x0, z0):
    # Vetor inicial concatenado
    y0 = np.array(x0 + z0)
    
    # Chama o fsolve para encontrar os zeros da função de resíduos
    sol = fsolve(system_residuals, y0, args=(u0, plenum_sys))
    
    # Separa a solução em x e z
    x_ss = sol[:n_x]
    z_ss = sol[n_x:]
    return x_ss, z_ss

if __name__ == '__main__':
    Phi, eta, Mach, Gimp, G2, Gdif, PHI, G2s, k = compression.character(compressor, m = 10, N = 750, Gi_1 = gas)
    x0 = [10, G2.T, G2.V]
    # E z0 pode ser (conforme seu código original):
    z0 = [G2.P, G2.P, Gimp.T, Gimp.V,
          Gdif.T, Gdif.V, G2s.T, G2s.V,
          G2.T, G2.V]+ [gas.V]
    
    # Vetor de parâmetros (u0) conforme seu modelo
    u0 = [4500, 300, 750, (10/k/(G2.P - 5000)**0.5), 5000]
    
    # Calcula os estados estacionários
    x_ss, z_ss = compute_steady_state(u0, gas, compressor, valve, plenum_sys, x0, z0)
    print("Estado estacionário (variáveis diferenciais):", x_ss)
    print("Estado estacionário (variáveis algébricas):", z_ss)

Estado estacionário (variáveis diferenciais): [1.00000000e+01 3.26515453e+02 2.90504717e-01]
Estado estacionário (variáveis algébricas): [8.45260762e+03 8.45260762e+03 3.14202300e+02 3.60706821e-01
 3.13821964e+02 3.68227009e-01 3.24481538e+02 2.87838547e-01
 3.26515453e+02 2.90504717e-01 5.09879009e-01]


C:\Users\lastb\AppData\Local\Temp\ipykernel_6332\782087007.py:39: RuntimeWarning: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
  sol = fsolve(system_residuals, y0, args=(u0, plenum_sys))


In [13]:
def test_evaluate_dae(mixture, volumn_desviation, vlv, Vpp, Lc, A1, gas, comp, visc, compressor, plenum_sys):

    dotm_0 = vlv.evaluate_flow((10/k/(G2.P - 5000)**0.5),2500)

    # Definição de valores numéricos de teste
    x_test = [1.00000000e+01, 3.26515448e+02, 2.90504791e-01]
    z_test = [8.45260542e+03, 8.45260542e+03, 3.14202300e+02, 3.60706821e-01,
 3.13821964e+02, 3.68227009e-01, 3.24481532e+02, 2.87838621e-01,
 3.26515448e+02, 2.90504791e-01, 5.09879009e-01]
    u_test = [4500, 300, 750, (10/k/(G2.P - 5000)**0.5), 5000]  # Parâmetros de entrada

    # Avaliação da função
    ode_values, alg_values = plenum_sys.evaluate_dae(None, x_test, z_test, u_test)

    # Imprimindo os resultados
    print("Valores das ODEs:")
    for i, val in enumerate(ode_values):
        print(f"ODE {i}: {val}")

    print("\nValores das equações algébricas:")
    for i, val in enumerate(alg_values):
        print(f"Algebrica {i}: {val}")

test_evaluate_dae(mixture, volumn_desviation, vlv, Vpp, Lc, A1, gas, comp, visc, compressor, plenum_sys)

Valores das ODEs:
ODE 0: [[3.03351422]]
ODE 1: [[3.51032137]]
ODE 2: [[-0.01806043]]

Valores das equações algébricas:
Algebrica 0: -1.241500285686925e-05
Algebrica 1: -1.241500285686925e-05
Algebrica 2: [[-4.81865541e-09]]
Algebrica 3: 1.612335893392182e-09
Algebrica 4: [[-1.92844772e-08]]
Algebrica 5: -4.91024043097332e-09
Algebrica 6: [[196.89076633]]
Algebrica 7: [[2.80985057]]
Algebrica 8: [[-393.83553971]]
Algebrica 9: 0.00421447938788333
Algebrica 10: 3.401589310669806e-06


In [ ]:
def run_simulation():
    list_names = ["CH4", "C2H6", "C3H8", "iC4H10", "nC4H10", "iC5H12", "nC5H12", 
                  "nC6H14", "nC7H16", "nC8H18", "nC9H20", "nC10H22", "nC11H24", 
                  "nC12H26", "nC14H30", "N2", "H2O", "CO2", "C15+"]

    nwe_3 = [0.9834, 0.0061, 0.0015, 0.0003, 0.0003, 0.00055, 0.0004, 0.0, 0.0, 0.0,
             0.0, 0.0, 0.0, 0.0, 0.0, 0.0003, 0.0, 0.0008, 0.0]

    dict_composition_3 = {list_names[i]: nwe_3[i] for i in range(len(nwe_3))}
    mixture_nwe_3 = Mixture(list_of_species, dict_composition_3)

    volumn_desviation = [0] * 19
    gas = gc_eos_class(mixture_nwe_3, 300, 4500, None, 1, 0, Aij, volumn_desviation, 'gas')  # T inicial em Kelvin

    comp = CompressorClass()
    visc = viscosity(mixture_nwe_3, volumn_desviation)
    compressor = compression(gas, comp, visc)

    vlv = valve(kv=0.38)
    Vpp = 2.0 
    Lc = 2.0 
    A1 = 2.6e-3
    plenum_sys = plenum(gas, compressor, vlv, Vpp, Lc, A1)

    dotm_0 = vlv.evaluate_flow((10/k/np.sqrt(G2.P - 5000)), 2500)

    # Chutes iniciais (x0: variáveis diferenciais, z0: variáveis algébricas)
    x0 = [1.00000000e+01, 3.26515453e+02, 2.90504717e-01]
    z0 = [8.45260762e+03, 8.45260762e+03, 3.14202300e+02, 3.60706821e-01,
 3.13821964e+02, 3.68227009e-01, 3.24481538e+02, 2.87838547e-01,
 3.26515453e+02, 2.90504717e-01, 5.09879009e-01]
    u0 = [4500, 300, 750, (10/k/np.sqrt(G2.P - 5000)), 5000]

    # Definição dos símbolos para o DAE
    x_sym = SX.sym('x', 3)
    z_sym = SX.sym('z', 11)
    u_sym = SX.sym('u', 5)

    ode_sym, alg_sym = plenum_sys.evaluate_dae(None, x_sym, z_sym, u_sym)

    # total_fun = ca.Function('f',[x_sym,z_sym,u_sym],ode_sym+alg_sym)

    # Fun = lambda y : [f.full().flatten().item() for f in total_fun(y[0:3],y[3:],u0)]
    # print(Fun())
    dae = {
        'x': x_sym,
        'z': z_sym,
        'p': u_sym,
        'ode': vertcat(*ode_sym),
        'alg': vertcat(*alg_sym)
    }

    t0 = 1
    tf = 100
    # Chamada atualizada do integrador com t0 e tf como argumentos
    integrator_solver = integrator('F', 'idas', dae, 0, 0.1)

    t_sim = 10
    n_steps = int(t_sim / tf)
    time = np.zeros(n_steps + 1)
    dot_m_hist = np.zeros(n_steps + 1)
    Tp_hist = np.zeros(n_steps + 1)
    Pp_hist = np.zeros(n_steps + 1)

    x0_current, z0_current = x0.copy(), z0.copy()
    res = integrator_solver(x0=x0_current, z0=z0_current, p=u0)



    # for i in range(n_steps + 1):
    #     if i > 0:
    #         res = integrator_solver(x0=x0_current, z0=z0_current, p=u0)
    #         x0_current = res['xf'].full().flatten()
    #         z0_current = res['zf'].full().flatten()
    #     time[i] = i * tf
    #     dot_m_hist[i] = x0_current[0]
    #     Tp_hist[i] = x0_current[1]
    #     Pp_hist[i] = z0_current[0]

    # plt.figure(figsize=(10, 8))
    # plt.subplot(3, 1, 1)
    # plt.plot(time, dot_m_hist, 'b-')
    # plt.ylabel('Fluxo de Massa [kg/s]')
    # plt.grid(True)
    
    # plt.subplot(3, 1, 2)
    # plt.plot(time, Tp_hist, 'r-')
    # plt.ylabel('Temperatura [K]')
    # plt.grid(True)
    
    # plt.subplot(3, 1, 3)
    # plt.plot(time, Pp_hist / 1e5, 'g-')
    # plt.ylabel('Pressão [bar]')
    # plt.xlabel('Tempo [s]')
    # plt.grid(True)
    
    # plt.tight_layout()
    # plt.savefig("grafico.png", dpi=300)


run_simulation()